In [1]:
import os
os.chdir('..')
os.chdir('..')
import scvelo as scv
import scanpy as sc
import cell2fate as c2f
import pickle as pickle
from eval_utils import cross_boundary_correctness
from datetime import datetime
import pandas as pd
import numpy as np
from os.path import exists
import matplotlib.pyplot as plt
import torch
import unitvelo as utv
method = 'Cell2fateDynamicalModel_TimeMixture'
data_dir = '/nfs/team283/aa16/data/fate_benchmarking/benchmarking_datasets/'
save_dir = '/nfs/team283/aa16/data/fate_benchmarking/benchmarking_results/'
datasets = ['Pancreas_with_cc',  'DentateGyrus' , 'MouseBoneMarrow', 'MouseErythroid', 'HumanBoneMarrow']
n_genes_list = np.array((2000, 3000))
n_counts_list = np.array((10, 20))

for i in range(len(n_genes_list)):
    for j in range(len(n_counts_list)):
        for k in range(len(datasets)):
            print(i)
            print(j)
            print(k)
            dataset = datasets[k]
            n_genes = n_genes_list[i]
            min_counts = n_counts_list[j]
            model_index = str(i) + '-' + str(j) + '-' + str(k)
            save_name = method + '_'
            if exists(save_dir + save_name + '_CBDC_fullBenchmark.csv'):
                tab = pd.read_csv(save_dir + save_name + '_CBDC_fullBenchmark.csv', index_col = 0)
                if model_index in tab.index:
                    continue
            adata = sc.read_h5ad(data_dir + dataset + '/' + dataset + '_anndata.h5ad')
            adata = c2f.utils.get_training_data(adata, cells_per_cluster = 10**6, cluster_column = 'clusters',
                                            remove_clusters = [], min_shared_counts = min_counts, n_var_genes= n_genes)
            c2f.Cell2fate_DynamicalModel_TimeMixture.setup_anndata(adata, spliced_label='spliced', unspliced_label='unspliced')    
            n_modules = c2f.utils.get_max_modules(adata)
            mod = c2f.Cell2fate_DynamicalModel_TimeMixture(adata,
                                               n_modules = n_modules)   
            mod.train()
            sample_kwarg = {"num_samples": 3, "batch_size" : adata.n_obs,
                 "use_gpu" : True, 'return_samples': False}
            adata = mod.export_posterior(adata, sample_kwargs = sample_kwarg)
            mod.compute_and_plot_total_velocity_scvelo(adata, save = False, delete = False)
            # Calculate performance metrics:
            file = open(data_dir + dataset + '/' + dataset + '_groundTruth.pickle' ,'rb')
            ground_truth = pickle.load(file)
            metrics = utv.evaluate(adata, ground_truth, 'clusters', 'velocity')
            cb_score = [np.mean(metrics['Cross-Boundary Direction Correctness (A->B)'][x])
                        for x in metrics['Cross-Boundary Direction Correctness (A->B)'].keys()]
            if exists(save_dir + save_name + '_CBDC_fullBenchmark.csv'):
                tab = pd.read_csv(save_dir + save_name + '_CBDC_fullBenchmark.csv', index_col = 0)
            else:
                c_names = ['CBDC']
                tab = pd.DataFrame(columns = c_names)
            tab.loc[model_index, 'CBDC'] = np.mean(cb_score)
            tab.to_csv(save_dir + save_name + '_CBDC_fullBenchmark.csv')
tab = pd.read_csv(save_dir + save_name + '_CBDC_fullBenchmark.csv', index_col = 0)
tab.loc['AVERAGE', 'CBDC'] = np.mean(tab['CBDC'])
tab.to_csv(save_dir + save_name + '_CBDC_fullBenchmark.csv')

Global seed set to 0


(Running UniTVelo 0.2.5)
2023-10-21 13:16:56
0
0
0
Keeping at most 1000000 cells per cluster
Filtered out 19641 genes that are detected 10 counts (shared).
Extracted 2000 highly variable genes.
Leiden clustering ...
Number of Leiden Clusters: 13
Maximal Number of Modules: 14


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Sampling local variables, batch:   0%|                                                                                                                                                                                 | 0/1 [00:00<?, ?it/s]



KeyboardInterrupt



In [1]:
import scvelo as scv
import scanpy as sc
import cell2fate as c2f
import pickle as pickle
from eval_utils import cross_boundary_correctness
from datetime import datetime
import pandas as pd
import numpy as np
from os.path import exists
import matplotlib.pyplot as plt
import torch
import unitvelo as utv
method = 'Cell2fateDynamicalModel_TimeMixture'
data_dir = '/nfs/team283/aa16/data/fate_benchmarking/benchmarking_datasets/'
save_dir = '/nfs/team283/aa16/data/fate_benchmarking/benchmarking_results/'
datasets = ['Pancreas_with_cc',  'DentateGyrus' , 'MouseBoneMarrow', 'MouseErythroid', 'HumanBoneMarrow']
n_genes_list = np.array((2000, 3000))
n_counts_list = np.array((10, 20))
i = 0
j = 0
k = 1

Global seed set to 0


(Running UniTVelo 0.2.5)
2023-10-21 16:46:18


In [ ]:
dataset = datasets[k]
n_genes = n_genes_list[i]
min_counts = n_counts_list[j]
model_index = str(i) + '-' + str(j) + '-' + str(k)
save_name = method + '_'
adata = sc.read_h5ad(data_dir + dataset + '/' + dataset + '_anndata.h5ad')
adata = c2f.utils.get_training_data(adata, cells_per_cluster = 10**6, cluster_column = 'clusters',
                                remove_clusters = [], min_shared_counts = min_counts, n_var_genes= n_genes)
c2f.Cell2fate_DynamicalModel_TimeMixture.setup_anndata(adata, spliced_label='spliced', unspliced_label='unspliced')    
n_modules = c2f.utils.get_max_modules(adata)
mod = c2f.Cell2fate_DynamicalModel_TimeMixture(adata, n_modules = n_modules)   
mod.train(max_epochs = 500)
sample_kwarg = {"num_samples": 3, "batch_size" : adata.n_obs,
     "use_gpu" : True, 'return_samples': False}
adata = mod.export_posterior(adata, sample_kwargs = sample_kwarg)
mod.compute_and_plot_total_velocity_scvelo(adata, save = False, delete = False)
# Calculate performance metrics:
file = open(data_dir + dataset + '/' + dataset + '_groundTruth.pickle' ,'rb')
ground_truth = pickle.load(file)
metrics = utv.evaluate(adata, ground_truth, 'clusters', 'velocity')
cb_score = [np.mean(metrics['Cross-Boundary Direction Correctness (A->B)'][x])
            for x in metrics['Cross-Boundary Direction Correctness (A->B)'].keys()]

Keeping at most 1000000 cells per cluster
Filtered out 9174 genes that are detected 10 counts (shared).
Extracted 2000 highly variable genes.
Leiden clustering ...
         Falling back to preprocessing with `sc.pp.pca` and default params.
Number of Leiden Clusters: 14
Maximal Number of Modules: 16


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Sampling global variables, sample: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  8.95it/s]
Computing total RNAvelocity ...


  0%|          | 0/2930 [00:00<?, ?cells/s]

In [ ]:
adata.obs['cell_weights'] = np.array(np.argmax(mod.samples['post_sample_means']['cell_weights'], axis = -1)).flatten()

In [ ]:
fig, ax = plt.subplots(1,1, figsize = (6, 4))
sc.pl.umap(adata, color = ['cell_weights'], s = 200, legend_loc='on data', show = False, ax = ax)

In [ ]:
fig, ax = plt.subplots(1,1, figsize = (6, 4))
sc.pl.umap(adata, color = ['Time (hours)'], s = 200, legend_loc='on data', show = False, ax = ax)